In [84]:
from elasticsearch import Elasticsearch
import pandas as pd


# Read the CSV file and store it as a DataFrame object
df = pd.read_csv('books_info.csv')
# Print the first 5 rows of the DataFrame
print(df.head())


                                               title  \
0  Goliath Joe, fisherman / by Charles Thackeray ...   
1  Grammar of the Narrinyeri tribe of Australian ...   
2                 The works of the Rev. Sydney Smith   
3  Nellie Doran : a story of Australian home and ...   
4  Lastkraftwagen 3 t Ford : Baumuster V 3000 S :...   

                                      url                    contributors  \
0  https://trove.nla.gov.au/work/10013347              Thackeray, Charles   
1  https://trove.nla.gov.au/work/10029401                  Taplin, George   
2   https://trove.nla.gov.au/work/1004403        Smith, Sydney, 1771-1845   
3  https://trove.nla.gov.au/work/10049667                   Miriam Agatha   
4  https://trove.nla.gov.au/work/10053234  Germany. Heer. Heereswaffenamt   

        date                                        format  \
0  1900-1919                         Book|Book/Illustrated   
1  1878-1880                   Book|Government publication   
2  1839-1900  

In [85]:
# split the values in col1 into two columns using the '-' separator
df[['start_date', 'finish_date']] = df['date'].str.split('-', n=1, expand=True)
print(df)


                                                   title  \
0      Goliath Joe, fisherman / by Charles Thackeray ...   
1      Grammar of the Narrinyeri tribe of Australian ...   
2                     The works of the Rev. Sydney Smith   
3      Nellie Doran : a story of Australian home and ...   
4      Lastkraftwagen 3 t Ford : Baumuster V 3000 S :...   
...                                                  ...   
42169  The "Nile" of Australia : Nature's gateway to ...   
42170  Aviation / delivered from 4BC Station, Brisban...   
42171                     Stigmata : a novel / Bill Reed   
42172  The vine in Australia / [A.C. Kelly]. Biograph...   
42173  Technological change in Australia / report of ...   

                                          url  \
0      https://trove.nla.gov.au/work/10013347   
1      https://trove.nla.gov.au/work/10029401   
2       https://trove.nla.gov.au/work/1004403   
3      https://trove.nla.gov.au/work/10049667   
4      https://trove.nla.gov.au/wo

In [86]:
# define a function to replace None with data from another column
def fill_none(row):
    if pd.isnull(row['finish_date']):
        return row['start_date']
    else:
        return row['finish_date']

# apply the function to the 'finish_date' column and update the dataframe
df['finish_date'] = df.apply(fill_none, axis=1)
df = df.drop('date', axis=1)

# remove rows with null values in the 'Name' column
df = df.dropna(subset=['start_date'], how='any')
df = df[df['start_date'].str.len() >= 4]

# drop all rows with NaN values
df = df.fillna('')

# for i, row in df.iterrows():
#     print(row)


In [87]:
from elasticsearch import Elasticsearch
import csv
# Initialize Elasticsearch client
es = Elasticsearch(hosts="http://localhost:9200", verify_certs=False)
# Define function to create Elasticsearch index and mapping
index_name = "books"
mapping = {
    "properties": {
        "title": {"type": "text"},
        "url": {"type": "keyword"},
        "contributors": {"type": "text"},
        "start_date": {"type": "date" ,"format": "yyyy"},
        "finish_date": {"type": "date" ,"format": "yyyy"},
        "format": {"type": "keyword"},
        "full_text_url": {"type": "keyword"},
        "trove_id": {"type": "keyword"},
        "language": {"type": "keyword"},
        "rights": {"type": "keyword"},
        "pages": {"type": "integer"},
        "form": {"type": "keyword"},
        "volume": {"type": "keyword"},
        "parent": {"type": "keyword"},
        "children": {"type": "keyword"},
        "text_downloaded": {"type": "keyword"},
        "text_file": {"type": "keyword"},
    }
}
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)

es.indices.create(index=index_name, body={"mappings": mapping})
print(es)
# Add the data to Elasticsearch
for i, row in df.iterrows():
    es.index(index=index_name, id=i, body=row.to_dict())

/var/folders/7n/b_wg4kps235fftnbjl1rv9680000gn/T/ipykernel_57229/3274040508.py:28: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  if es.indices.exists(index=index_name):
/var/folders/7n/b_wg4kps235fftnbjl1rv9680000gn/T/ipykernel_57229/3274040508.py:29: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.indices.delete(index=index_name)
/var/folders/7n/b_wg4kps235fftnbjl1rv9680000gn/T/ipykernel_57229/3274040508.py:31: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.indices.creat

<Elasticsearch(['http://localhost:9200'])>


In [88]:
res = es.search(index="books", body={"query":{"bool": { "must_not": { "match": {"parent": ""} } }}, "size":10000})
print(res)


/var/folders/7n/b_wg4kps235fftnbjl1rv9680000gn/T/ipykernel_57229/595060596.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res = es.search(index="books", body={"query":{"bool": { "must_not": { "match": {"parent": ""} } }}, "size":10000})


{'took': 1985, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1422, 'relation': 'eq'}, 'max_score': 0.0, 'hits': [{'_index': 'books', '_type': '_doc', '_id': '254', '_score': 0.0, '_source': {'title': 'Abstract and analysis of the evidence taken by a Select Committee of the Legislative Assembly of Victoria, in the case of George Dunmore Lang, late Manager, and Frederic Lee Drake, late accountant, of the branch Bank of New South Wales at Ballaarat : with explanatory notes and observations, together with reports on the evidence ... ..', 'url': 'https://trove.nla.gov.au/work/11179694', 'contributors': 'Lang, John Dunmore, 1799-1878', 'format': 'Book|Article/Other article|Article', 'fulltext_url': 'http://nla.gov.au/nla.obj-412255342', 'trove_id': 'nla.obj-412255342', 'language': '', 'rights': '', 'pages': 104, 'form': 'Multi Volume Book', 'volume': 1.0, 'parent': 'nla.obj-412253504', 'children': '', 'text_downloaded': T

/var/folders/7n/b_wg4kps235fftnbjl1rv9680000gn/T/ipykernel_57229/595060596.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  res = es.search(index="books", body={"query":{"bool": { "must_not": { "match": {"parent": ""} } }}, "size":10000})


In [89]:
from cassandra.cluster import Cluster

cluster = Cluster(['127.0.0.1']) 
session = cluster.connect()
session.execute("CREATE KEYSPACE IF NOT EXISTS book_cassandra1 WITH replication = {'class': 'SimpleStrategy', 'replication_factor': '1'}")

session.execute("DROP TABLE IF EXISTS book_cassandra1.child")
session.execute("CREATE TABLE IF NOT EXISTS book_cassandra1.child (id text, title text, contributors text, fulltext_url text, format text,start_date text,pages int, PRIMARY KEY (id))")

for hit in res['hits']['hits']:
    print(hit)
    session.execute(
        """
        INSERT INTO book_cassandra1.child (id,title,contributors,fulltext_url,format,start_date,pages)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        """,
        (hit['_source']['trove_id'], hit['_source']['title'], hit['_source']['contributors'], hit['_source']['fulltext_url'], hit['_source']['format'], hit['_source']['start_date'], hit['_source']['pages'])
    )

{'_index': 'books', '_type': '_doc', '_id': '254', '_score': 0.0, '_source': {'title': 'Abstract and analysis of the evidence taken by a Select Committee of the Legislative Assembly of Victoria, in the case of George Dunmore Lang, late Manager, and Frederic Lee Drake, late accountant, of the branch Bank of New South Wales at Ballaarat : with explanatory notes and observations, together with reports on the evidence ... ..', 'url': 'https://trove.nla.gov.au/work/11179694', 'contributors': 'Lang, John Dunmore, 1799-1878', 'format': 'Book|Article/Other article|Article', 'fulltext_url': 'http://nla.gov.au/nla.obj-412255342', 'trove_id': 'nla.obj-412255342', 'language': '', 'rights': '', 'pages': 104, 'form': 'Multi Volume Book', 'volume': 1.0, 'parent': 'nla.obj-412253504', 'children': '', 'text_downloaded': True, 'text_file': 'abstract-and-analysis-of-the-evidence-taken-by-a-s-nla.obj-412255342.txt', 'start_date': '1857', 'finish_date': '1857'}}
{'_index': 'books', '_type': '_doc', '_id': 

In [90]:
resAdult = es.search(index="books", body={"query":{"bool": { "must_not": { "match": {"children": ""} } }}, "size":10000})
print(resAdult)

{'took': 20, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 239, 'relation': 'eq'}, 'max_score': 0.0, 'hits': [{'_index': 'books', '_type': '_doc', '_id': '256', '_score': 0.0, '_source': {'title': 'Abstract and analysis of the evidence taken by a Select Committee of the Legislative Assembly of Victoria, in the case of George Dunmore Lang, late Manager, and Frederic Lee Drake, late accountant, of the branch Bank of New South Wales at Ballaarat : with explanatory notes and observations, together with reports on the evidence ... ..', 'url': 'https://trove.nla.gov.au/work/11179694', 'contributors': 'Lang, John Dunmore, 1799-1878', 'format': 'Book|Article/Other article|Article', 'fulltext_url': 'http://nla.gov.au/nla.obj-412253504', 'trove_id': 'nla.obj-412253504', 'language': '', 'rights': '', 'pages': 0, 'form': 'Multi Volume Book', 'volume': '', 'parent': '', 'children': 'nla.obj-412255342|nla.obj-75124485', 'text_dow

/var/folders/7n/b_wg4kps235fftnbjl1rv9680000gn/T/ipykernel_57229/3644785998.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  resAdult = es.search(index="books", body={"query":{"bool": { "must_not": { "match": {"children": ""} } }}, "size":10000})
/var/folders/7n/b_wg4kps235fftnbjl1rv9680000gn/T/ipykernel_57229/3644785998.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  resAdult = es.search(index="books", body={"query":{"bool": { "must_not": { "match": {"children": ""} } }}, "size":10000})


In [91]:
session.execute("DROP TABLE IF EXISTS book_cassandra1.adultBook")
session.execute("CREATE TABLE IF NOT EXISTS book_cassandra1.adultBook (id text, title text, contributors text, fulltext_url text, format text,start_date text,pages int, PRIMARY KEY (id))")

for hit in resAdult['hits']['hits']:
    print(hit)
    session.execute(
        """
        INSERT INTO book_cassandra1.adultBook (id,title,contributors,fulltext_url,format,start_date,pages)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        """,
        (hit['_source']['trove_id'], hit['_source']['title'], hit['_source']['contributors'], hit['_source']['fulltext_url'], hit['_source']['format'], hit['_source']['start_date'], hit['_source']['pages'])
    )

{'_index': 'books', '_type': '_doc', '_id': '256', '_score': 0.0, '_source': {'title': 'Abstract and analysis of the evidence taken by a Select Committee of the Legislative Assembly of Victoria, in the case of George Dunmore Lang, late Manager, and Frederic Lee Drake, late accountant, of the branch Bank of New South Wales at Ballaarat : with explanatory notes and observations, together with reports on the evidence ... ..', 'url': 'https://trove.nla.gov.au/work/11179694', 'contributors': 'Lang, John Dunmore, 1799-1878', 'format': 'Book|Article/Other article|Article', 'fulltext_url': 'http://nla.gov.au/nla.obj-412253504', 'trove_id': 'nla.obj-412253504', 'language': '', 'rights': '', 'pages': 0, 'form': 'Multi Volume Book', 'volume': '', 'parent': '', 'children': 'nla.obj-412255342|nla.obj-75124485', 'text_downloaded': False, 'text_file': '', 'start_date': '1857', 'finish_date': '1857'}}
{'_index': 'books', '_type': '_doc', '_id': '273', '_score': 0.0, '_source': {'title': "The Burke and

In [92]:
# Execute CQL query to get the average number of pages
result = session.execute("SELECT AVG(pages) FROM book_cassandra1.child;")

# Iterate over the result set and print the average number of pages
for row in result:
    print("Average number of pages: {}".format(row[0]))

Average number of pages: 298


In [93]:
# Execute CQL query to get the average number of pages
result = session.execute("SELECT AVG(pages) FROM book_cassandra1.adultBook;")

# Iterate over the result set and print the average number of pages
for row in result:
    print("Average number of pages: {}".format(row[0]))

Average number of pages: 0


In [94]:
query = """
        SELECT *
        FROM book_cassandra1.adultBook
        WHERE start_date < '2000' 
        ALLOW FILTERING;
        """
result = session.execute(query)

for row in result:
    print((row))

Row(id='nla.obj-33236945', contributors='Society for Nautical Research (London, England)', format='Book|Book/Illustrated', fulltext_url='http://nla.gov.au/nla.obj-33236945', pages=0, start_date='1935', title='Lists of men-of-war, 1650-1700')
Row(id='nla.obj-39267340', contributors='Campbell, Bessie, 1870-1964', format='Book', fulltext_url='https://nla.gov.au/nla.obj-39267340', pages=0, start_date='1889', title='[Bessie Campbell, Banjo Queen : a collection of music programmes, ephemera relating to her career]')
Row(id='nla.obj-475664029', contributors='', format='Government publication|Book', fulltext_url='http://nla.gov.au/nla.obj-475664029', pages=0, start_date='1860', title='Report of the proceedings of the Queensland Government schooner "Spitfire" in search of the mouth of the River Burdekin, on the north-eastern coast of Australia : and of the exploration of a portion of that coast extending from Gloucester Island to Halifax Bay')
Row(id='nla.obj-410740363', contributors='', format

In [95]:
query = """
        SELECT *
        FROM book_cassandra1.child
        WHERE start_date < '2000' 
        ALLOW FILTERING;
        """
result = session.execute(query)

for row in result:
    print((row))

Row(id='nla.obj-56155448', contributors='', format='Book', fulltext_url='http://nla.gov.au/nla.obj-56155448', pages=600, start_date='1716', title='Kangxi zi dian')
Row(id='nla.obj-56180948', contributors='', format='Book', fulltext_url='http://nla.gov.au/nla.obj-56180948', pages=600, start_date='1716', title='Kangxi zi dian')
Row(id='nla.obj-57729010', contributors='', format='Book', fulltext_url='http://nla.gov.au/nla.obj-57729010', pages=64, start_date='1800', title='Zhong xiao jie yi er du mei zhuan')
Row(id='nla.obj-56186140', contributors='', format='Book', fulltext_url='http://nla.gov.au/nla.obj-56186140', pages=600, start_date='1716', title='Kangxi zi dian')
Row(id='nla.obj-56155644', contributors='', format='Book', fulltext_url='http://nla.gov.au/nla.obj-56155644', pages=600, start_date='1716', title='Kangxi zi dian')
Row(id='nla.obj-56173548', contributors='', format='Book', fulltext_url='http://nla.gov.au/nla.obj-56173548', pages=600, start_date='1716', title='Kangxi zi dian'